imports & preprocessing

In [6]:
!pip install opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.0/30.0 MB 34.3 MB/s eta 0:00:0000:0100:01


In [3]:
%load_ext autoreload
%autoreload 1
import tensorflow as tf
from tensorflow import keras
from keras.layers import (Conv2D, MaxPool2D, Dropout, Flatten, 
    Dense, BatchNormalization, RandomFlip, RandomContrast, RandomRotation, RandomZoom,LeakyReLU)
from keras.optimizers import Adam
from keras.models import Sequential
from keras.losses import SparseCategoricalCrossentropy
from keras.metrics import SparseCategoricalAccuracy
import os
import cv2
from sklearn.model_selection import train_test_split
import numpy as np
import preprocessing
from model import CNN_Model
import recall


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
import cv2

ModuleNotFoundError: No module named 'cv2'

In [2]:
#TODO: change path to absolute path of data/images folder
training_data, testing_data, training_labels, testing_labels = preprocessing.generate_data(testing_size = 0.2, image_height=256, image_width=256, path="/Users/juliannerudner/Desktop/CS1430/CS1430_Final_Project/data/images") 

NameError: name 'preprocessing' is not defined

In [5]:
model = CNN_Model()
architecture = Sequential([
                Conv2D(16, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Conv2D(16, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Conv2D(32, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Conv2D(32, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Dropout(0.1), 
                Conv2D(64, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Conv2D(64, (2,2), padding='same'),
                LeakyReLU(),
                BatchNormalization(),
                MaxPool2D(pool_size=(2,2)),
                Flatten(),
                Dense(128, activation='relu'),
                BatchNormalization(),
                Dense(64, activation='relu'),
                BatchNormalization(),
                Dropout(0.1), 
                Dense(21, activation="softmax")
            ])
checkpoint_path = "training_1/cp.ckpt" #TODO: change to desired path
checkpoint_dir = os.path.dirname(checkpoint_path)
os.listdir(checkpoint_dir)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.build_model(architecture=architecture, optimizer=Adam, learning_rate=0.002)
# model.train(train_data=training_data,train_labels=training_labels,augment=False,epochs=20,batch_size=10,callbacks=cp_callback)


2022-12-12 10:47:42.602525: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
checkpoint_path = "checkpoints/acc_65/cp.ckpt" #TODO: change to desired path
model.load_state(checkpoint_path)
test_loss, test_accuracy = model.test(test_data=testing_data,test_labels=testing_labels)
print("done testing")

6/6 [==============================] - 4s 426ms/step - loss: 0.5957 - sparse_categorical_accuracy: 0.8063
done testing


Below is a method for retrieving the predicted classes (by id) after training the model. We can use this to compare to the actual labels / retrieve recall information

In [7]:
predicted_classes = np.argmax(model.model.predict(testing_data,batch_size=15),axis=1)
print(predicted_classes)
print(testing_labels)
def acc(predicted_classes, testing_labels):
    a = 0
    for i in range(len(predicted_classes)):
        if predicted_classes[i]==testing_labels[i]:
            a+=1
    acc = a / len(predicted_classes)
    return acc

print(acc(predicted_classes, testing_labels))

13/13 [==============================] - 3s 177ms/step
[ 0  0 17 17  9 12  1 11  2 18  2 18 16  3  3  3  4 11  4  4  4 18 13  6
 17  6  6  6  6  6 18  6  6  7  7  7  7 11 17 20  8  9 20  6  9 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10  6 10 10 11 11 11 11 11 11
 11 18 11 11 11 11 16 12  6 11 12 13 13 13 13  6 13 14 14 14 14 14 14 14
 14 15 15 15 15 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16
 16 16 16 17 17 17 17 17 17 17  6 17 11 18 17 17  7 17 17 18 18 18 18 18
 18 18 18 18 18 18 18 18 18 18 18 18 19 13 17 19 19 20 20 20 11 20 11 20
 20 11 20 20 20 20 20 20 11 20 20 20 20 11 20 20 11 20 20 20 20 20 20]
[ 0  0  0  0  1  1  1  1  2  2  2  2  3  3  3  3  4  4  4  4  4  5  5  5
  5  6  6  6  6  6  6  6  6  7  7  7  7  8  8  8  8  9  9  9  9 10 10 10
 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 11 11 11 11 11 11
 11 11 11 11 11 11 11 12 12 12 12 13 13 13 13 13 13 14 14 14 14 14 14 14
 14 15 15 15 15 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 16 1

Below is code for seeing if any of the identified medications have been recalled

In [16]:
data = recall.loaddata()
for pill in predicted_classes:
    is_recalled = data[' Recalled'][pill] #TODO: replace this with the recall.recall function
    if is_recalled:
        print(data['Name'][pill])